# Notebook for a space-time discretization

In this notebook we are treating following problem 
$$
u_t - \Delta u + \mathbf{w} \cdot \nabla u = f \quad \text{in } \Omega  = [0,1] \times [0,T].
$$
with the initial/boundary conditions
$$
u(x,0) = \sin(\pi x), \quad u(0,t) = u(1,t)=0.
$$
The weakformulation of the problem in the DG setting is

Find a function $u_0 \in H^1_0(\Omega)$ , such that  for all  $v \in H^1_0(\Omega) $:

$$
\int_{\Omega} (u_t)_0 v \,dx \,dt
+ \int_{\Omega} \nabla u_0 \cdot \nabla v \,dx \,dt
+ \int_{\Omega} (\mathbf{w} \cdot \nabla u_0) v \,dx \,dt
= \int_{\Omega} f v \,dx \,dt
- \int_{\Omega} u_D v \,dx \,dt.
$$

where we used the decomposition of $u = u_D + u_0$

For $w = 0$ (the heatequation), we have for the chosen initial/boundary conditions following analytical solution 

$u(x,t)= \sin(\pi x) e^{-t \pi^2}$

In [ ]:
from ngsolve import *
from netgen.geom2d import SplineGeometry
from ngsolve.webgui import Draw
from netgen.occ import *

T = 0.5 # length of the time interval
shape = Rectangle(1,T).Face()
shape.edges.Min(X).name="left"
shape.edges.Max(X).name="right"
shape.edges.Min(Y).name="bottom"
shape.edges.Max(Y).name="top"
mesh = Mesh(OCCGeometry(shape, dim=2).GenerateMesh(maxh=0.05))
Draw(mesh)

In [ ]:
order=6
fes = L2(mesh, order=order, dgjumps=True)
u,v = fes.TnT()
w = 5
lam = 2
h = specialcf.mesh_size

dS = dx(skeleton=True) 
jump_u = u-u.Other()
jump_v = v-v.Other()
n = specialcf.normal(2)
avgu = 0.5*n[0] * (grad(u)[0]+grad(u.Other())[0])
avgv= 0.5*n[0] * (grad(v)[0]+grad(v.Other())[0])

Au = CoefficientFunction((grad(u)[0],0))
B = CoefficientFunction((w,1))

#diff = BilinearForm(fes, symmetric=False)
diff = Au*grad(v)*dx # diffusion term
diff += -avgu * jump_v *dS #term from the partial integration
diff += (B*grad(u))*v*dx # time derivative term
#a += lam*order**2/h*u*v*ds(skeleton=True)#weakly impose u=sin(pi*x) on the bottom boundary
diff += -(n[0]*grad(u)[0]*v)* ds(skeleton=True) #term from the partial integration
diff += lam*order**2/h*u*v * ds(skeleton=True) # penalty term
diff += lam*order**2/h*jump_u*jump_v*dS # penalty term
a = BilinearForm(diff).Assemble()

f = LinearForm(fes)
f += lam*order**2/h*exp(w*x/2)*sin(pi*x)*v*ds(skeleton=True, definedon=mesh.Boundaries("bottom"))#weakly impose boundary condition
f.Assemble()

gfu = GridFunction(fes)

gfu.vec.data += a.mat.Inverse(fes.FreeDofs()) * f.vec
Draw(gfu)

l2error = sqrt(Integrate((gfu-(sin(pi*x)*exp(-(pi**2+w**2/4)*y)*exp(w*x/2)))**2, mesh))
#l2error = sqrt(Integrate((gfu-(sin(pi*x)*exp(-pi**2*y)))**2, mesh))
print(l2error)


In [ ]:
diff += - B * grad(u)*v*dx
uhat = IfPos(B*n, u, u.Other())
con = -B*grad(v)*u*dx
con += (B*n)*uhat *jump_v*dS
diffcon =BilinearForm(diff + con).Assemble()

gfu1 = GridFunction(fes)
gfu1.vec.data += diffcon.mat.Inverse(fes.FreeDofs()) * f.vec
Draw(gfu1)

l2error = sqrt(Integrate((gfu-(sin(pi*x)*exp(-(pi**2+w**2/4)*y)*exp(w*x/2)))**2, mesh))

#l2error = sqrt(Integrate((gfu1-(sin(pi*x)*exp(-pi**2*y)))**2, mesh))
print(l2error)
